In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
df = pd.read_json('sample_dataset/batch_log_s.json', lines=True)

In [3]:
df.head(10)

,D,T,amount,event_type,id,id1,id2,timestamp
0,2.0,50.0,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,56.69,purchase,1.0,NaN,NaN,2017-06-14 18:46:40
2,NaN,NaN,20.68,purchase,2.0,NaN,NaN,2017-06-14 18:46:40
3,NaN,NaN,27.91,purchase,3.0,NaN,NaN,2017-06-14 18:46:40
4,NaN,NaN,76.61,purchase,4.0,NaN,NaN,2017-06-14 18:46:40
5,NaN,NaN,56.01,purchase,5.0,NaN,NaN,2017-06-14 18:46:40
6,NaN,NaN,73.72,purchase,6.0,NaN,NaN,2017-06-14 18:46:40
7,NaN,NaN,24.11,purchase,7.0,NaN,NaN,2017-06-14 18:46:40
8,NaN,NaN,33.43,purchase,8.0,NaN,NaN,2017-06-14 18:46:40
9,NaN,NaN,49.89,purchase,9.0,NaN,NaN,2017-06-14 18:46:40


In [4]:
df.describe()

,D,T,amount,id,id1,id2
count,1.0,1.0,10.00000,10.00000,13.000000,13.000000
mean,2.0,50.0,44.33000,5.50000,4.769231,6.461538
std,NaN,NaN,21.04408,3.02765,2.803386,3.230464
min,2.0,50.0,20.68000,1.00000,1.000000,2.000000
25%,2.0,50.0,25.16500,3.25000,3.000000,3.000000
50%,2.0,50.0,41.66000,5.50000,4.000000,6.000000
75%,2.0,50.0,56.52000,7.75000,7.000000,10.000000
max,2.0,50.0,76.61000,10.00000,9.000000,10.000000


In [5]:
for possible_value in set(df['event_type'].tolist()):
    print(possible_value)

nan
unfriend
befriend
purchase


In [6]:
df_purchase=df[df['event_type']=='purchase']
df_purchase = df_purchase[['event_type','id','timestamp','amount']]
df_purchase.head()

,event_type,id,timestamp,amount
1,purchase,1.0,2017-06-14 18:46:40,56.69
2,purchase,2.0,2017-06-14 18:46:40,20.68
3,purchase,3.0,2017-06-14 18:46:40,27.91
4,purchase,4.0,2017-06-14 18:46:40,76.61
5,purchase,5.0,2017-06-14 18:46:40,56.01


In [7]:
df_befriend=df[df['event_type']=='befriend']
df_befriend=df_befriend[['event_type','id1','id2','timestamp']]
df_befriend.head()

,event_type,id1,id2,timestamp
11,befriend,1.0,2.0,2017-06-14 18:46:40
12,befriend,1.0,3.0,2017-06-14 18:46:40
13,befriend,6.0,10.0,2017-06-14 18:46:30
14,befriend,1.0,8.0,2017-06-14 18:46:40
15,befriend,3.0,6.0,2017-06-14 18:46:40


In [8]:
df_unfriend=df[df['event_type']=='unfriend']
df_unfriend=df_unfriend[['event_type','id1','id2','timestamp']]
df_unfriend.head()

,event_type,id1,id2,timestamp
23,unfriend,6.0,10.0,2017-06-14 18:46:40


In [9]:
df_friend = pd.concat([df_befriend, df_unfriend])
df_friend.tail()
df_friend = df_friend.sort_values('timestamp')
df_friend

,event_type,id1,id2,timestamp
13,befriend,6.0,10.0,2017-06-14 18:46:30
11,befriend,1.0,2.0,2017-06-14 18:46:40
12,befriend,1.0,3.0,2017-06-14 18:46:40
14,befriend,1.0,8.0,2017-06-14 18:46:40
15,befriend,3.0,6.0,2017-06-14 18:46:40
16,befriend,4.0,5.0,2017-06-14 18:46:40
17,befriend,7.0,3.0,2017-06-14 18:46:40
18,befriend,8.0,6.0,2017-06-14 18:46:40
19,befriend,8.0,9.0,2017-06-14 18:46:40
20,befriend,9.0,2.0,2017-06-14 18:46:40


In [10]:
G = nx.Graph()

In [11]:
id1list = df_friend.id1.tolist()
id2list = df_friend.id2.tolist()
idlist = set(id1list + id2list)
G.add_nodes_from(idlist)
list(G.nodes())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [12]:
N = df_friend.shape[0]
for i in range(N):
    id10 = df_friend.iloc[i].id1
    id20 = df_friend.iloc[i].id2
    event_type0 = df_friend.iloc[i].event_type
    if event_type0 == 'befriend':
        G.add_edge(id10,id20)
    if event_type0 == 'unfriend':
        if G.has_edge(id10,id20):
            G.remove_edge(id10,id20)

list(G.edges())

[(1.0, 2.0),
 (1.0, 3.0),
 (1.0, 8.0),
 (2.0, 9.0),
 (3.0, 6.0),
 (3.0, 7.0),
 (4.0, 5.0),
 (4.0, 10.0),
 (6.0, 8.0),
 (8.0, 9.0)]

In [13]:
df_friend[df_friend['id1'] == 1.0]

,event_type,id1,id2,timestamp
11,befriend,1.0,2.0,2017-06-14 18:46:40
12,befriend,1.0,3.0,2017-06-14 18:46:40
14,befriend,1.0,8.0,2017-06-14 18:46:40


In [14]:
df_DT=df[df['D'].notnull()]
df_DT=df_DT[['D','T']]
D = df_DT.values[0][0]
T = df_DT.values[0][1]
print(D)
print(T)
df_DT.head()

2.0
50.0


,D,T
0,2.0,50.0


In [15]:
df_DT.values[0][1]

50.0

In [16]:
G[10.0]

{4.0: {}}

In [17]:
G.number_of_nodes()

10

In [18]:
G.number_of_edges()

10

In [19]:
list(G.edges())

[(1.0, 2.0),
 (1.0, 3.0),
 (1.0, 8.0),
 (2.0, 9.0),
 (3.0, 6.0),
 (3.0, 7.0),
 (4.0, 5.0),
 (4.0, 10.0),
 (6.0, 8.0),
 (8.0, 9.0)]

In [20]:
list(G.neighbors(1.0))

[2.0, 3.0, 8.0]

In [21]:
list(G.neighbors(3.0))

[1.0, 6.0, 7.0]

In [22]:
nnode = 1.0

Dlist = list(G.neighbors(nnode))
print(Dlist)
for i in list(G.neighbors(nnode)):
    print(list(G.neighbors(i)))
    Dlist += list(G.neighbors(i))
    print(Dlist)
    print(len(list(G.neighbors(i))))

print(Dlist)
print(set(Dlist))
print(len(set(Dlist)))



[2.0, 3.0, 8.0]
[1.0, 9.0]
[2.0, 3.0, 8.0, 1.0, 9.0]
2
[1.0, 6.0, 7.0]
[2.0, 3.0, 8.0, 1.0, 9.0, 1.0, 6.0, 7.0]
3
[1.0, 6.0, 9.0]
[2.0, 3.0, 8.0, 1.0, 9.0, 1.0, 6.0, 7.0, 1.0, 6.0, 9.0]
3
[2.0, 3.0, 8.0, 1.0, 9.0, 1.0, 6.0, 7.0, 1.0, 6.0, 9.0]
{1.0, 2.0, 3.0, 6.0, 7.0, 8.0, 9.0}
7


In [23]:
len(list(nx.ego_graph(G, nnode, 2)))
# the number is different????

7

In [33]:
AA = set(list(nx.ego_graph(G, nnode, 2)))
len(AA)

7

In [34]:
userid = df_purchase[df_purchase['id'] == nnode]
print(len(userid))
if len(userid) > T:
    userid = userid.sort_values('timestamp')[-T:]

sum0 = userid.amount.sum()
mean0 = userid.amount.mean()
sd0 = userid.amount.std()

print("sum: {0:.2f} mean: {1:.2f} sd: {2:.2f}".format(sum0, mean0, sd0))
print('sum: ' + str(sum0) + ' mean: ' + str(mean0) + ' sd: ' + str(sd0))

1
sum: 56.69 mean: 56.69 sd: nan
sum: 56.69 mean: 56.69 sd: nan


In [26]:
for i in np.arange(D):
    print(i)
    
list(nx.ego_graph(G, 1.0, 1))
print(len(list(nx.ego_graph(G, 1.0, 1))))

0.0
1.0
4


In [27]:
#nx.draw(G)

In [28]:
for possible_value in set(df_purchase['timestamp'].tolist()):
    print(possible_value)

2017-06-14 18:46:40


In [29]:
for possible_value in set(df_befriend['timestamp'].tolist()):
    print(possible_value)

2017-06-14 18:46:40
2017-06-14 18:46:30


In [30]:
for possible_value in set(df_unfriend['timestamp'].tolist()):
    print(possible_value)

2017-06-14 18:46:40


In [31]:
G.degree(1)

3

In [32]:
print(len(G.neighbors(1)))
list(G.neighbors(1))

3


[2.0, 3.0, 8.0]